This is the main Sensor Kennel application that will 
identify, read and plot any sensor that's plugged in.

On the lower side:  Pin1 (left) Gnd, Pin3 3v3, Pin-3 (from right) SDA, Pin-5 SCL.

In [43]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [44]:
print(i2c.scan())

[57, 60]


In [28]:
%sendtofile --source SI7021_funcs.py


Sent 55 lines (2042 bytes) to SI7021_funcs.py.


In [103]:
%sendtofile --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --binary --source OLED_driver.py

Sent 55 lines (2042 bytes) to SI7021_funcs.py.
Sent 33 lines (1131 bytes) to TSL561_funcs.py.
Sent 25 lines (779 bytes) to device_detection.py.
Sent 3084 bytes in 103 chunks to OLED_driver.py.


In [104]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext

import time
time.sleep(0.5)
from device_detection import IdentifyI2CDevice

devices = IdentifyI2CDevice(i2c)
devname = "nothing"
for dev in devices:
    devname, devdesc = dev.split()
    fbuff.fill(0)
    fbuff.text(devname,0,0,1)
    fbuff.text(devdesc,0,12,1)
    doublepixels()
    oledshow()
    time.sleep(2)
    
if devname == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        oledshow()

if devname == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist: %.1f"%d, 8, 20, 1)
        oledshow()

if devname == "TSL561":
    from TSL561_funcs import setupTSL561, luminosity
    setupTSL561(i2c)
    while True:
        br, ir = luminosity()
        fbuff.fill(0)
        fbuff.text("TSL561", 0, 0, 1)
        fbuff.text("broad:", 0, 23, 1)
        fatntext("%d"%br, 50, 16)
        fbuff.text("ir:", 0, 53, 1)
        fatntext("%d"%ir, 50, 46)
        oledshow()


Sent 52 lines (1410 bytes) to main.py.


In [95]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [57]:
help(i2c)

object <I2C> is of type I2C
  init -- <function>
  scan -- <function>
  start -- <function>
  stop -- <function>
  readinto -- <function>
  write -- <function>
  readfrom -- <function>
  readfrom_into -- <function>
  writeto -- <function>
  readfrom_mem -- <function>
  readfrom_mem_into -- <function>
  writeto_mem -- <function>


In [96]:
chip_id = i2c.readfrom_mem(0x39, 0x80 | 0x0A, 1)[0]
partno = (chip_id >> 4) & 0x0f  # should be 5
revno = chip_id & 0x0f
print(partno, revno)
#chip_id = self._read_register(_REGISTER_ID)
#        partno = (chip_id >> 4) & 0x0f
#        revno = chip_id & 0x0f
#        return (partno, revno)


5 0


In [99]:
i2c.writeto_mem(0x39, 0x80 | 0x00, b"\x03")  # _CONTROL_POWERON


In [88]:
x = i2c.readfrom_mem(0x39, 0x80 | 0x01, 1)[0]  # _REGISTER_TIMING
print(hex(x))

0x12


In [87]:
i2c.writeto_mem(0x39, 0x80 | 0x01, chr(x | 0x10))


In [102]:
import ustruct
def luminosity():
    br = ustruct.unpack("<h", i2c.readfrom_mem(0x39, 0x80 | 0x0C, 2))[0]
    ir = ustruct.unpack("<h", i2c.readfrom_mem(0x39, 0x80 | 0x0E, 2))[0]
    return br, ir  # (can't fetch both with same read command)
for i in range(10):
    print(luminosity())
    time.sleep(1)
    

(494, 160)
(492, 159)
(491, 159)
(63, 23)
(446, 143)
(497, 160)
(488, 157)
(489, 157)
(489, 158)
(491, 158)
.

In [48]:

# pylint: disable=bad-whitespace
_DEFAULT_ADDRESS     = const(0x39)
_COMMAND_BIT         = const(0x80)
_WORD_BIT            = const(0x20)

_CONTROL_POWERON     = const(0x03)
_CONTROL_POWEROFF    = const(0x00)

_REGISTER_CONTROL    = const(0x00)
_REGISTER_TIMING     = const(0x01)
_REGISTER_CHAN0_LOW  = const(0x0C)
_REGISTER_CHAN1_LOW  = const(0x0E)
_REGISTER_ID         = const(0x0A)

_GAIN_SCALE = (16, 1)
_TIME_SCALE = (1 / 0.034, 1 / 0.252, 1)
_CLIP_THRESHOLD = (4900, 37000, 65000)
# pylint: enable=bad-whitespace

    @property
    def lux(self):
        """The computed lux value."""
        return self._compute_lux()

    @property
    def broadband(self):
        """The broadband channel value."""
        return self._read_broadband()

    @property
    def infrared(self):
        """The infrared channel value."""
        return self._read_infrared()

    @property
    def luminosity(self):
        """The overall luminosity as a tuple containing the broadband
        channel and the infrared channel value."""
        return (self.broadband, self.infrared)

    @property
    def integration_time(self):
        """The integration time. 0:13.7ms, 1:101ms, 2:402ms, or 3:manual"""
        current = self._read_register(_REGISTER_TIMING)
        return current & 0x03

    @integration_time.setter
    def integration_time(self, value):
        """Set the integration time. 0:13.7ms, 1:101ms, 2:402ms, or 3:manual."""
        value &= 0x03
        current = self._read_register(_REGISTER_TIMING)
        self.buf[0] = _COMMAND_BIT | _REGISTER_TIMING
        self.buf[1] = (current & 0xfc) | value
        with self.i2c_device as i2c:
            i2c.write(self.buf, end=2)

    def _compute_lux(self):
        """Based on datasheet for FN package."""
        ch0, ch1 = self.luminosity
        if ch0 == 0:
            return None
        if ch0 > _CLIP_THRESHOLD[self.integration_time]:
            return None
        if ch1 > _CLIP_THRESHOLD[self.integration_time]:
            return None
        ratio = ch1 / ch0
        if ratio >= 0 and ratio <= 0.50:
            lux = 0.0304 * ch0 - 0.062 * ch0 * ratio**1.4
        elif ratio <= 0.61:
            lux = 0.0224 * ch0 - 0.031 * ch1
        elif ratio <= 0.80:
            lux = 0.0128 * ch0 - 0.0153 * ch1
        elif ratio <= 1.30:
            lux = 0.00146 * ch0 - 0.00112 * ch1
        else:
            lux = 0
        # Pretty sure the floating point math formula on pg. 23 of datasheet
        # is based on 16x gain and 402ms integration time. Need to scale
        # result for other settings.
        # Scale for gain.
        lux *= _GAIN_SCALE[self.gain]
        # Scale for integration time.
        lux *= _TIME_SCALE[self.integration_time]
        return lux

    def _enable(self):
        self._write_control_register(_CONTROL_POWERON)

    def _disable(self):
        self._write_control_register(_CONTROL_POWEROFF)

    def _read_register(self, reg, count=1):
        self.buf[0] = _COMMAND_BIT | reg
        if count == 2:
            self.buf[0] |= _WORD_BIT
        i2c.write(self.buf, end=1, stop=False)
        i2c.readinto(self.buf, start=1)
        if count == 1:
            return self.buf[1]
        elif count == 2:
            return self.buf[1], self.buf[2]
        return None

    def _write_control_register(self, reg):
        self.buf[0] = _COMMAND_BIT | _REGISTER_CONTROL
        self.buf[1] = reg
        i2c.write(self.buf, end=2)

    def _read_broadband(self):
        low, high = self._read_register(_REGISTER_CHAN0_LOW, 2)
        return high << 8 | low

    def _read_infrared(self):
        low, high = self._read_register(_REGISTER_CHAN1_LOW, 2)
        return high << 8 | low


In [49]:
t = TSL2561(i2c)


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 26, in __init__
  File "<stdin>", line 35, in chip_id
  File "<stdin>", line 145, in _read_register
TypeError: function does not take keyword arguments


In [56]:
import array
buf = array.array("b", [1,2,3])
buf = bytearray(3)
print(buf)
print(i2c.readfrom_into(0x39, buf))
print(buf)


bytearray(b'\x00\x00\x00')
None
bytearray(b'\x00\x02\x00')
